# FAQ ChatBot model prototype notebook

## 1. Project Overview

The aim of this notebook is to use asymmetric semantic search to retrieve the top 3 closest answers from the ReactJS FAQ dataset based on a user's question input. Questions and answers are mapped to the same vector space using a pre-trained SBERT model. Given a new question, the closest 3 answers are retrieved using similarity search. The dataset will be split, with 90% used as reference corpus and 10% of questions reserved to evaluate how well the system retrieves relevant answers.

## 2. Dataset

The ReactJS FAQ dataset will be used, which consists approximately 26,300 question-answer pairs in a JSON format.

In [9]:
import pandas as pd
import numpy as np

# Import display module from IPython to render pandas DataFrames in VS Code notebooks
# from IPython.display import display # uncomment if needed to display multiple objects in one cell

In [ ]:
# Load dataset from kaggle

import kagglehub
from kagglehub import KaggleDatasetAdapter

# JSON file inside the ReactJS FAQ dataset folder
file_path = "React_dataset.json"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "savanidhruv/reactjs-faq-dataset",
  file_path,
)

print("First 5 records:")
df.head()

First 5 records:


,question,answer,detailedanswer,detailed_answer,argument
0,What is React?,React is an open-source JavaScript library for...,NaN,NaN,NaN
1,What is a Date Picker in React?,A Date Picker in React is a UI component that ...,NaN,NaN,NaN
2,what are feature of React?,"1) Component-Based Architecture, 2) Virtual DO...",NaN,NaN,NaN
3,Which libraries are commonly used for date pic...,"Popular libraries include react-datepicker, re...",NaN,NaN,NaN
4,How to install react-datepicker?,Run `npm install react-datepicker` and also in...,NaN,NaN,NaN


## 3. Preprocessing



## 4. Embedding Generation

## 5. Similarity Measure

## 6. Testing

## 7. Evaluation